In [48]:
import local_seqtools.general_utils as tools
import pandas as pd
import numpy as np
import local_conservation_analysis_pipeline.group_conservation_objects as group_tools
import json
import os
import re
import sys
from pathlib import Path
from Bio import AlignIO, Seq, SeqIO, Align

import matplotlib.pyplot as plt
plt.style.use('custom_standard')
# plt.style.use('custom_small')
import seaborn as sns
# pd.options.plotting.backend = "plotly"
from ast import literal_eval

# %load_ext autoreload
# %autoreload 2

In [49]:
zscores = pd.read_csv('./z_scores_wideform.csv')
score_key = pd.read_csv('./z_scores_score_key.csv')
score_key['reciprocal_best_match'] = score_key['reciprocal_best_match'].apply(lambda x: bool(literal_eval(x)) if x == 'True' or x == 'False' else x)
score_indexes=[i for i in zscores.columns if i in score_key['score_index'].unique()]

for col in score_indexes:
    zscores[col] = zscores[col].apply(literal_eval)

zscores['mask_array'] = zscores['mask_array'].str.replace(' ', ',')
zscores['mask_array'] = zscores['mask_array'].apply(literal_eval)
zscores['mask_array'] = zscores['mask_array'].apply(np.array)

def extract_important_scores(s, colname):
    scores = np.array(s[colname])
    maskarr = np.array(s["mask_array"])
    mask = maskarr.nonzero()
    return scores[mask]

zscores2=zscores.copy()
for i in score_indexes:
    zscores2[i] = zscores2.apply(lambda x: extract_important_scores(x, i), axis=1)
    zscores2[i] = zscores2[i].apply(np.mean)

# zscores2 = zscores2[zscores2['ELM_motif_class'] != 'TRAF6']
passing_ref_id_list = zscores['reference_index'].unique()

In [50]:
zscores['mask_array'].isnull().sum()

0

# supplemental table - counts for each set for each SLiM in the benchmark

In [51]:
supp_folder = Path('./supplement')
class_counts_folder = supp_folder / 'benchmark_class_counts'
class_counts_folder.mkdir(exist_ok=True, parents=True)
# zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack().to_csv(class_counts_folder / 'benchmark_class_counts.csv')
# zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack()
temp = zscores.groupby('verified interaction')['ELM_motif_class'].value_counts().unstack().T.copy()
re_map = zscores.groupby('ELM_motif_class').first()['regex'].to_dict()
mask_map = zscores.groupby('ELM_motif_class').first()['mask_array'].to_dict()
mask_map['LIG_14-3-3_CanoR_1'] = 'No mask'
temp['regex'] = temp.index.map(re_map)
temp['mask_array'] = temp.index.map(mask_map)
temp.to_csv(class_counts_folder / 'benchmark_class_counts.csv')
temp

verified interaction,False,True,regex,mask_array
ELM_motif_class,,,,
DOC_WW_Pin1_4,242,42,...([ST])P.,"[0, 0, 0, 1, 1, 0]"
Ena/VASP ([FL]PPPP),241,34,[FL]PPPP,"[1, 1, 0, 1, 1]"
LIG_14-3-3_CanoR_1,246,41,"R[^DE]{0,2}[^DEPG]([ST])(([FWYLMV].)|([^PRIKGN...",No mask
LIG_AP2alpha_2,241,49,DP[FW],"[1, 1, 1]"
LIG_EH_1,269,49,.NPF.,"[0, 1, 1, 1, 0]"
LIG_SH2_GRB2like,210,18,(Y)([EDST]|[MLIVAFYHQW])N.,"[1, 1, 1, 0]"
LIG_SH3_CIN85_PxpxPR_1,212,28,P.[AP].PR,"[1, 0, 1, 0, 1, 1]"
TRAF6,199,7,...P.E..[FYWDE],"[0, 0, 0, 1, 0, 1, 0, 0, 1]"


# supplementary table - conservation scores

In [52]:
score_indexes

['0.0',
 '2.0',
 '4.0',
 '8.0',
 '9.0',
 '12.0',
 '13.0',
 '21.0',
 '27.0',
 '35.0',
 '41.0',
 '44.0',
 '45.0',
 '48.0',
 '49.0',
 '52.0',
 '53.0',
 '56.0',
 '57.0',
 '60.0',
 '61.0',
 'kibby_hit_zscores']

In [53]:
score_key[(score_key['reciprocal_best_match']!=True)&(score_key['lflank']!='5.0')]

,score_index,aln_type,level,score_key,table_file,lflank,rflank,reciprocal_best_match,columnwise_score_function_name,similarity_threshold,num_passing
0,0.0,MSA - MAFFT,Tetrapoda,aln_property_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,property_entropy,MSA,2362
1,1.0,MSA - MAFFT,Tetrapoda,aln_shannon_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,shannon_entropy,MSA,2362
2,2.0,MSA - MAFFT,Vertebrata,aln_property_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,property_entropy,MSA,2524
3,3.0,MSA - MAFFT,Vertebrata,aln_shannon_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,shannon_entropy,MSA,2524
4,4.0,MSA - MAFFT,Metazoa,aln_property_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,property_entropy,MSA,2343
5,5.0,MSA - MAFFT,Metazoa,aln_shannon_entropy,/home/jch/Documents/07-pt1_pt2_combined/benchm...,MSA,MSA,MSA,shannon_entropy,MSA,2343
11,11.0,Pairwise,Tetrapoda,fragpair_gapless_lf0_rf0_edssmat50,/home/jch/Documents/07-pt1_pt2_combined/benchm...,0.0,0.0,False,shannon_entropy,1.0,2544
13,13.0,Pairwise,Tetrapoda,fragpair_gapless_lf0_rf0_edssmat50,/home/jch/Documents/07-pt1_pt2_combined/benchm...,0.0,0.0,False,property_entropy,1.0,2544
30,30.0,Pairwise,Vertebrata,fragpair_gapless_lf0_rf0_blosum62,/home/jch/Documents/07-pt1_pt2_combined/benchm...,0.0,0.0,False,shannon_entropy,1.0,2706
31,31.0,Pairwise,Vertebrata,fragpair_gapless_lf0_rf0_grantham,/home/jch/Documents/07-pt1_pt2_combined/benchm...,0.0,0.0,False,shannon_entropy,1.0,2706


In [54]:
# score_key_filt = score_key[score_key['score_index'].isin(score_indexes)]
# score_key_filt = score_key_filt[score_key_filt['score_index']!= 'kibby_hit_zscores']

In [61]:
cols2export=list(score_key[
    (score_key['reciprocal_best_match']!=True)&
    (~score_key['lflank'].isin(['5.0','10.0','2.0']))&
    (score_key['columnwise_score_function_name']!='shannon_entropy')&
    (~score_key['score_key'].isin(['fragpair_gapless_lf0_rf0_blosum62','fragpair_gapless_lf0_rf0_grantham', 'kibby']))
].score_index.unique())
cols2export
score_key_for_rename = score_key[score_key['score_index'].isin(cols2export)].copy()
score_key_for_rename['aln_type'] = score_key_for_rename['aln_type'].replace('Pairwise', 'pairk')
score_key_for_rename['aln_type'] = score_key_for_rename['aln_type'].replace('Pairwise embedding', 'pairk - ESM2 embeddings')

In [62]:
zscores_export = zscores2[
    [
        'ELM_motif_class',
        'verified interaction',
        'regex',
        'UniprotID',
        'gene_id',
        'hit start position',
        'hit end position',
        'hit_sequence',
        'mask_array',
    ]
    + cols2export
    + ['kibby_hit_zscores']
].copy()
rn_dict = {
    'ELM_motif_class': 'SLiM class',
    'gene_id': 'OrthoDB id',
    'kibby_hit_zscores': 'Kibby',
}
zscores_export = zscores_export.rename(columns=rn_dict)

def score_index_2_col_title(score_index, score_key_df):
    df = score_key_df[score_key_df['score_index']==score_index].copy()
    assert len(df)==1, f"More than one match for {score_index}"
    level=df['level'].values[0]
    aln_type=df['aln_type'].values[0]
    title = f'{aln_type} - {level}'
    return title

rn_dict2 ={}
for i in cols2export:
    rn_dict2[i] = score_index_2_col_title(i, score_key_for_rename)
zscores_export = zscores_export.rename(columns=rn_dict2)
zscores_export

,SLiM class,verified interaction,regex,UniprotID,OrthoDB id,hit start position,hit end position,hit_sequence,mask_array,MSA - MAFFT - Tetrapoda,MSA - MAFFT - Vertebrata,MSA - MAFFT - Metazoa,pairk - Tetrapoda,pairk - Vertebrata,pairk - Metazoa,pairk - ESM2 embeddings - Tetrapoda,pairk - ESM2 embeddings - Vertebrata,pairk - ESM2 embeddings - Metazoa,Kibby
0,LIG_AP2alpha_2,True,DP[FW],O08838,10116_0:004cdf,356,358,DPF,"[1, 1, 1]",1.312759,0.865163,-0.906534,0.765409,0.387204,-0.484739,1.047175,0.779145,-0.120440,1.727165
1,LIG_AP2alpha_2,True,DP[FW],Q05140,10116_0:002e5e,399,401,DPF,"[1, 1, 1]",0.180739,-1.107886,-1.174238,0.284105,0.369749,0.344503,0.547614,0.943751,0.766580,2.232242
2,LIG_AP2alpha_2,True,DP[FW],Q05140,10116_0:002e5e,473,475,DPF,"[1, 1, 1]",-0.091241,1.006945,0.850580,0.284105,0.369749,0.344503,0.547614,0.939865,0.753180,2.818305
3,LIG_AP2alpha_2,True,DP[FW],P98082,9606_0:0016b2,292,294,DPF,"[1, 1, 1]",1.385546,1.510597,0.930359,0.907574,1.022667,1.213079,1.233372,1.392867,1.850673,1.770583
4,LIG_AP2alpha_2,True,DP[FW],P98082,9606_0:0016b2,297,299,DPF,"[1, 1, 1]",1.432202,1.150243,0.472766,0.907574,1.022667,1.213079,1.233372,1.174462,1.721987,1.799091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,TRAF6,False,...P.E..[FYWDE],A0A384MEH2,9606_0:00423d,921,929,LGAPEEPQE,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",-0.247717,-0.227983,-1.317094,-0.344617,-0.632294,-1.297816,-0.002785,-0.284336,-0.608505,-0.234538
2124,TRAF6,False,...P.E..[FYWDE],A0A384MDR3,9606_0:004627,81,89,IKKPAEDEW,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",-0.052190,0.005508,0.246967,-0.018752,-0.117315,-0.239458,-0.002198,-0.096858,-0.062096,0.649855
2125,TRAF6,False,...P.E..[FYWDE],Q14209,9606_0:00055f,363,371,SLVPLEATD,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",-0.580748,-0.738502,-1.153336,0.612861,0.338816,-0.064088,0.448872,0.086616,-0.064803,0.675122
2126,TRAF6,False,...P.E..[FYWDE],A4PB67,9606_0:00057e,215,223,FHQPEEEIE,"[0, 0, 0, 1, 0, 1, 0, 0, 1]",-2.319332,-1.760295,-1.513717,-0.174233,0.002682,0.247917,-0.608636,-0.337345,-0.388173,0.031406


In [63]:
zscores_export=zscores_export.rename(columns={'hit start position': 'match start position', 'hit end position': 'match end position', 'hit_sequence': 'match sequence'})

In [64]:
output_dir=supp_folder / 'benchmark_zscores'
output_dir.mkdir(exist_ok=True, parents=True)
zscores_export.to_csv(output_dir / 'benchmark_scores_v2.csv', index=False)

In [58]:
zscores_export['mask_array'].values[0]

array([1, 1, 1])